In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [3]:
train = pd.read_csv('../data/original_data/train.csv')
test = pd.read_csv('../data/original_data/test.csv')
combine = [train, test]

In [4]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [5]:
test.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [6]:
print(len(train))
print(len(test))

891
418


In [7]:
print('+'*40)
train.info()
print('='*40)
test.info()
print('+'*40)

++++++++++++++++++++++++++++++++++++++++
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passenger

In [8]:
print(204 / 891 * 100)
print(91 / 418 * 100)

22.895622895622896
21.770334928229666


- Cabin 데이터는 트레인, 테스트 모두 결측률이 80% 이므로 사용하지 않는다.

In [9]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [10]:
train.describe(include=["O"])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Abelson, Mr. Samuel",male,1601,C23 C25 C27,S
freq,1,577,7,4,644


- Ticket, Cabin 을 제외한 나머지 열들로 부터 생존율을 구해본다.

In [11]:
train[["Pclass","Survived"]].groupby("Pclass").mean()

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [12]:
train[["Sex","Survived"]].groupby("Sex").mean()

,Survived
Sex,
female,0.742038
male,0.188908


In [13]:
train[["SibSp","Survived"]].groupby("SibSp").mean()

,Survived
SibSp,
0,0.345395
1,0.535885
2,0.464286
3,0.250000
4,0.166667
5,0.000000
8,0.000000


- SibSp는 5와 8이 구분이 불가능
- 가공이 필요한 데이터

In [14]:
train[["Parch","Survived"]].groupby("Parch").mean()

,Survived
Parch,
0,0.343658
1,0.550847
2,0.500000
3,0.600000
4,0.000000
5,0.200000
6,0.000000


- 4와 6이 구분 불가능
- 가공이 필요한 데이터

In [15]:
train[["Fare","Survived"]].groupby("Fare").mean()

,Survived
Fare,
0.0000,0.066667
4.0125,0.000000
5.0000,0.000000
6.2375,0.000000
6.4375,0.000000
...,...
227.5250,0.750000
247.5208,0.500000
262.3750,1.000000


- 연속적인 데이터에 해당하므로 적절한 범주화가 필요함

- Age는 연속적데이터 결측값 있으므로 Nan 처리 후 범주화 필요
- Embarked는 Nan 처리 필요

### Age 추정
1. 전체데이터의 평균, 분산 을 구해서 난수발생 집어넣기 -> 정확하지 않음
2. 특정 범주에서 난수혹은 평균을 찾아서 대체
3. 1과 2를 동시에 사용

In [16]:
guess_ages = np.zeros((2, 3))

In [17]:
Sex_mapping = {
    'female' : 1,
    'male'   : 2
}

for dataset in combine:
    dataset['Sex'] = dataset["Sex"].map(Sex_mapping)

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",2,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",2,35.0,0,0,373450,8.0500,NaN,S


In [18]:
for dataset in combine:
    for i in [1, 2]: # sex
        for j in [1, 2, 3]: # Pclass
            guessing_data = dataset[
                (dataset["Sex"] == i) &\
                (dataset["Pclass"] == j)
            ]["Age"].dropna()
            
            guess_ages[i-1, j-1] = int(guessing_data.median()/0.5 +0.5) * 0.5
    
    for i in [1, 2]: # sex
        for j in [1, 2, 3]: # Pclass
            dataset.loc[
                (dataset["Sex"] == i) & \
                (dataset["Pclass"] == j) &\
                (dataset["Age"].isnull()),
                "Age"
            ] = guess_ages[i-1, j-1]
    dataset["Age"] = dataset["Age"].astype(int)
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",2,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",2,35,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",2,27,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",1,19,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,21,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",2,26,0,0,111369,30.0000,C148,C


In [19]:
for dataset in combine:
    dataset["Title"] = dataset.Name.str.extract(" ([A-Za-z]+)\.")

In [20]:
train.Title.value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Major         2
Mlle          2
Col           2
Jonkheer      1
Countess      1
Capt          1
Don           1
Lady          1
Sir           1
Mme           1
Ms            1
Name: Title, dtype: int64

In [21]:
test.Title.value_counts()

Mr        240
Miss       78
Mrs        72
Master     21
Col         2
Rev         2
Ms          1
Dr          1
Dona        1
Name: Title, dtype: int64

In [22]:
def Title_mapping(x):
    if x in ["Dr","Rev","Col","Major","Don","Capt",
             "Jonkheer","Sir","Countess","Lady","Dona"]:
        return "Rare"
    return x

for dataset in combine:
    dataset["Title"] = dataset["Title"].map(Title_mapping)
    
    dataset["Title"] = dataset["Title"].replace("Ms",  "Miss")
    dataset["Title"] = dataset["Title"].replace("Mlle","Miss")
    
    dataset["Title"] = dataset["Title"].replace("Mme", "Mrs")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",2,22,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",2,35,0,0,373450,8.0500,NaN,S,Mr


In [23]:
train[["Title","Survived"]].groupby("Title").mean()

,Survived
Title,
Master,0.575000
Miss,0.702703
Mr,0.156673
Mrs,0.793651
Rare,0.347826


In [24]:
print((train.shape,test.shape))
for dataset in combine:
    dataset.drop(["Name","Ticket","Cabin"], axis = 1, inplace = True)
print((train.shape,test.shape))

((891, 13), (418, 12))
((891, 10), (418, 9))


In [25]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,2,22,1,0,7.2500,S,Mr
1,2,1,1,1,38,1,0,71.2833,C,Mrs
2,3,1,3,1,26,0,0,7.9250,S,Miss
3,4,1,1,1,35,1,0,53.1000,S,Mrs
4,5,0,3,2,35,0,0,8.0500,S,Mr
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,2,27,0,0,13.0000,S,Rare
887,888,1,1,1,19,0,0,30.0000,S,Miss
888,889,0,3,1,21,1,2,23.4500,S,Miss
889,890,1,1,2,26,0,0,30.0000,C,Mr


In [26]:
train["AgeBand"] = pd.cut(train.Age, 5)
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,1,0,3,2,22,1,0,7.2500,S,Mr,"(16.0, 32.0]"
1,2,1,1,1,38,1,0,71.2833,C,Mrs,"(32.0, 48.0]"
2,3,1,3,1,26,0,0,7.9250,S,Miss,"(16.0, 32.0]"
3,4,1,1,1,35,1,0,53.1000,S,Mrs,"(32.0, 48.0]"
4,5,0,3,2,35,0,0,8.0500,S,Mr,"(32.0, 48.0]"
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,2,27,0,0,13.0000,S,Rare,"(16.0, 32.0]"
887,888,1,1,1,19,0,0,30.0000,S,Miss,"(16.0, 32.0]"
888,889,0,3,1,21,1,2,23.4500,S,Miss,"(16.0, 32.0]"
889,890,1,1,2,26,0,0,30.0000,C,Mr,"(16.0, 32.0]"


In [27]:
train[["AgeBand","Survived"]].groupby("AgeBand").mean()

,Survived
AgeBand,
"(-0.08, 16.0]",0.550000
"(16.0, 32.0]",0.337374
"(32.0, 48.0]",0.412037
"(48.0, 64.0]",0.434783
"(64.0, 80.0]",0.090909


In [28]:
for dataset in combine:
    dataset.loc[(dataset["Age"] <= 16), "Age"] = 0
    dataset.loc[(dataset["Age"] > 16) & (dataset["Age"] <= 32), "Age"] = 1
    dataset.loc[(dataset["Age"] > 32) & (dataset["Age"] <= 48), "Age"] = 2
    dataset.loc[(dataset["Age"] > 48) & (dataset["Age"] <= 64), "Age"] = 3
    dataset.loc[(dataset["Age"] > 64), "Age"] = 4
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,1,0,3,2,1,1,0,7.2500,S,Mr,"(16.0, 32.0]"
1,2,1,1,1,2,1,0,71.2833,C,Mrs,"(32.0, 48.0]"
2,3,1,3,1,1,0,0,7.9250,S,Miss,"(16.0, 32.0]"
3,4,1,1,1,2,1,0,53.1000,S,Mrs,"(32.0, 48.0]"
4,5,0,3,2,2,0,0,8.0500,S,Mr,"(32.0, 48.0]"


In [29]:
train.drop("AgeBand", axis = 1, inplace = True)

In [30]:
train["FareBand"] = pd.qcut(train["Fare"], 3)
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FareBand
0,1,0,3,2,1,1,0,7.2500,S,Mr,"(-0.001, 8.662]"
1,2,1,1,1,2,1,0,71.2833,C,Mrs,"(26.0, 512.329]"
2,3,1,3,1,1,0,0,7.9250,S,Miss,"(-0.001, 8.662]"
3,4,1,1,1,2,1,0,53.1000,S,Mrs,"(26.0, 512.329]"
4,5,0,3,2,2,0,0,8.0500,S,Mr,"(-0.001, 8.662]"
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,2,1,0,0,13.0000,S,Rare,"(8.662, 26.0]"
887,888,1,1,1,1,0,0,30.0000,S,Miss,"(26.0, 512.329]"
888,889,0,3,1,1,1,2,23.4500,S,Miss,"(8.662, 26.0]"
889,890,1,1,2,1,0,0,30.0000,C,Mr,"(26.0, 512.329]"


In [31]:
train[["FareBand","Survived"]].groupby("FareBand").mean()

,Survived
FareBand,
"(-0.001, 8.662]",0.198052
"(8.662, 26.0]",0.402778
"(26.0, 512.329]",0.559322


In [32]:
test.Fare.fillna(train.Fare.mean(), inplace = True)
for dataset in combine:
    dataset.loc[(dataset["Fare"] <= 8.662), "Fare"] = 0
    dataset.loc[(dataset["Fare"] > 8.662) & (dataset["Fare"] <= 26.0), "Fare"] = 1
    dataset.loc[(dataset["Fare"] > 26.0), "Fare"] = 2
    dataset.Fare = dataset.Fare.astype(int)
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FareBand
0,1,0,3,2,1,1,0,0,S,Mr,"(-0.001, 8.662]"
1,2,1,1,1,2,1,0,2,C,Mrs,"(26.0, 512.329]"
2,3,1,3,1,1,0,0,0,S,Miss,"(-0.001, 8.662]"
3,4,1,1,1,2,1,0,2,S,Mrs,"(26.0, 512.329]"
4,5,0,3,2,2,0,0,0,S,Mr,"(-0.001, 8.662]"


In [33]:
for dataset in combine:
    dataset["IsAlone"] = 0

for dataset in combine:
    dataset.loc[(dataset.SibSp == 0) & (dataset.Parch == 0), "IsAlone"] = 1
train.drop("FareBand", axis = 1, inplace = True)
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,IsAlone
0,1,0,3,2,1,1,0,0,S,Mr,0
1,2,1,1,1,2,1,0,2,C,Mrs,0
2,3,1,3,1,1,0,0,0,S,Miss,1
3,4,1,1,1,2,1,0,2,S,Mrs,0
4,5,0,3,2,2,0,0,0,S,Mr,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,2,1,0,0,1,S,Rare,1
887,888,1,1,1,1,0,0,2,S,Miss,1
888,889,0,3,1,1,1,2,1,S,Miss,0
889,890,1,1,2,1,0,0,2,C,Mr,1


In [34]:
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,IsAlone
0,892,3,2,2,0,0,0,Q,Mr,1
1,893,3,1,2,1,0,0,S,Mrs,0
2,894,2,2,3,0,0,1,Q,Mr,1
3,895,3,2,1,0,0,1,S,Mr,1
4,896,3,1,1,1,1,1,S,Mrs,0
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,2,1,0,0,0,S,Mr,1
414,1306,1,1,2,0,0,2,C,Rare,1
415,1307,3,2,2,0,0,0,S,Mr,1
416,1308,3,2,1,0,0,0,S,Mr,1


In [35]:
train.fillna("S", inplace =True)
train.Embarked.isnull().sum()

0

In [38]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [40]:
train.drop(["SibSp","Parch","PassengerId"], axis = 1, inplace = True)
test.drop(["SibSp","Parch"], axis = 1, inplace = True)
train

,Survived,Pclass,Sex,Age,Fare,IsAlone,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,0,3,2,1,0,0,0,0,1,0,0,1,0,0
1,1,1,1,2,2,0,1,0,0,0,0,0,1,0
2,1,3,1,1,0,1,0,0,1,0,1,0,0,0
3,1,1,1,2,2,0,0,0,1,0,0,0,1,0
4,0,3,2,2,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,2,1,1,1,0,0,1,0,0,0,0,1
887,1,1,1,1,2,1,0,0,1,0,1,0,0,0
888,0,3,1,1,1,0,0,0,1,0,1,0,0,0
889,1,1,2,1,2,1,1,0,0,0,0,1,0,0


In [41]:
X_train = train.drop("Survived", axis = 1)
Y_train = train["Survived"]
X_test = test.drop("PassengerId", axis = 1)

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
RFC = RandomForestClassifier()

In [44]:
RFC.fit(X_train,Y_train)

RandomForestClassifier()

In [45]:
acc = RFC.score(X_train, Y_train)
acc

0.8574635241301908

In [46]:
predict = RFC.predict(X_test)

In [47]:
data = {
    "PassengerId" : test["PassengerId"],
    "Survived"    : predict
}
pd.DataFrame(data).to_csv("yapppppppp.csv", index = 0)